In [19]:
import networkx as nx
from networkx.readwrite import json_graph
from networkx.algorithms import bipartite
import json

In [9]:
with open('all_eebo.json', 'r') as jsonfile:
    jsonstring = jsonfile.read()
    data = json.loads(jsonstring)

In [14]:
B = json_graph.node_link_graph(data)
print(nx.info(B))

Name: 
Type: Graph
Number of nodes: 41619
Number of edges: 51123
Average degree:   2.4567


In [17]:
people_nodes = {n for n, d in B.nodes(data=True) if d['bipartite']==1}
text_nodes = set(B) - people_nodes
print(people_nodes)

{'1032869', '1015350', '1030695', '1023334', '1010697', '1031144', '1027467', '1024207', '1025963', '1032780', '1033108', '1020331', '1018352', '1009646', '1024300', '1022354', '1021213', '1027062', '1012216', '1016395', '1011213', '1014711', '1020061', '1014549', '1025825', '1028030', '1028969', '1032348', '1022755', '1025170', '1032171', '1031039', '1015480', '1023911', '1003210', '1014129', '1027533', '1024988', '1020640', '1000214', '1033372', '1000062', '1032762', '1015336', '1013794', '1017400', '1029074', '1020523', '1030031', '1024388', '1025091', '1029724', '1032811', '1010190', '1028174', '1026848', '1010233', '1021102', '1015715', '1006920', '1032480', '1028491', '1006276', '1008465', '1032355', '1033525', '1014280', '1018902', '1019140', '1031591', '1020229', '1017508', '1016360', '1018697', '1027097', '1002937', '1011589', '1010609', '1027822', '1021395', '1003592', '1012969', '1009753', '1026549', '1021322', '1011897', '1025987', '1010643', '1005816', '1024192', '1020590'

In [26]:
P = bipartite.weighted_projected_graph(B, people_nodes)
print(nx.info(P))

Name: 
Type: Graph
Number of nodes: 29368
Number of edges: 311822
Average degree:  21.2355


In [37]:
T = bipartite.weighted_projected_graph(B, text_nodes)
print(nx.info(T))

Name: 
Type: Graph
Number of nodes: 12251
Number of edges: 4114227
Average degree: 671.6557


In [24]:
def write_json(B, filename):
    """
    Given NetworkX graph object and filename, create JSON representation for D3.
    """
    print("Writing JSON file...")
    new_data = json_graph.node_link_data(B)
    with open(filename, 'w') as output:
        json.dump(new_data, output, sort_keys=True, indent=4, separators=(',',':'))

In [25]:
write_json(P, 'people_only.json')

Writing JSON file...


In [41]:
# print(list(P.edges(data=True))[:100])
fewer_edges = {(s,t) for s,t,d in P.edges(data=True) if d['weight']>1}
print(len(fewer_edges))
PS = P.edge_subgraph(fewer_edges)
print(nx.info(PS))
write_json(PS, 'people_more_than_two.json')

9148
Name: 
Type: Graph
Number of nodes: 3078
Number of edges: 9148
Average degree:   5.9441
Writing JSON file...


In [39]:
text_fewer_edges = {(s,t) for s,t,d in T.edges(data=True) if d['weight']>2}
print(len(text_fewer_edges))
TS = T.edge_subgraph(text_fewer_edges)
print(nx.info(TS))
write_json(TS, 'texts_more_than_two.json')

6008
Name: 
Type: Graph
Number of nodes: 1130
Number of edges: 6008
Average degree:  10.6336
Writing JSON file...
